## Step 0: 生成“脏”数据

In [ ]:
# 环境依赖
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

In [ ]:
# 构建原始字典数据 (对应上午的学习内容：理解字典到DataFrame的转换)
data = {
    'Transaction_ID': [101, 102, 103, 104, 102, 105, 106, 107, 108, 109],  # 注意 102 重复了
    'Branch': ['Beijing_A', 'Shanghai_B', 'Beijing_A', 'Shenzhen_C', 'Shanghai_B', 
               'Beijing_A', 'Shenzhen_C', 'Shanghai_B', 'Beijing_A', 'Shenzhen_C'],
    'Product': ['Latte', 'Espresso', 'Cappuccino', 'Latte', 'Espresso', 
                'Mocha', 'Latte', 'Espresso', 'Latte', 'Mocha'],
    'Price': [30.0, 25.0, 32.0, 30.0, 25.0, np.nan, 30.0, 25.0, 30.0, np.nan], # 有缺失值
    'Quantity': [1, 2, 1, 3, 2, 2, 1, 5, 2, 1],
    'Date': ['2023-10-01'] * 10
}

# 创建 DataFrame
df_initial = pd.DataFrame(data)

# 保存为 CSV (对应中午的学习内容：数据加载)
df_initial.to_csv('dirty_coffee_sales.csv', index=False)

print("项目数据 'dirty_coffee_sales.csv' 已生成！")

## Step 1: 基础了解

In [ ]:
# 理解数据基本形态
ser_product = pd.Series(data=data['Product'])
print("任务 A: 手动创建的 Product 列 Series:")
ser_product

In [ ]:
# 使用 .loc 找出标签（索引）为 3 的那行数据
print("任务 B: 使用 .loc 找出标签为 3 的那行数据:")
df_initial.loc[3]

In [ ]:
# 使用 .iloc 找出前 5 行、前 2 列的数据
print("任务 B: 使用 .iloc 找出前 5 行、前 2 列的数据:")
df_initial.iloc[:5, :2]

## Step 2: 数据加载与预览

In [ ]:
df = pd.read_csv('dirty_coffee_sales.csv')
print("数据预览:head():")
display(df.head())
print("数据预览:info():")
display(df.info())

df

## Step 3: 数据清洗与处理

In [ ]:
print("任务 A: 处理重复值")
display(df.duplicated())
df = df.drop_duplicates().reset_index(drop=True)
display(df)

In [ ]:
# 处理缺失值
print("任务 B: 处理缺失值")
df = df.fillna({'Price': 35.0}) # 假设摩卡的价格为35.0 + 字典赋值改进
df.isna().sum() # 输出每个Series的缺失值数量

### fillna() 实践

这里我们一开始直接使用了 `df.fillna(35.0)`  
但是这会导致所有缺失值都被填充为 35.0  
显然不是我们想要的结果  
那么我们实际上是想要 `Price` 列的缺失值被填充为 35.0  
因此我们可以使用字典来指定每一列的填充值  
此外，这还支持多`Series`同时填充不同的值  

```python
df = df.fillna({'Price': 35.0})
```

另一种方法就是只填充和更新 `Price` 列的缺失值  

```python
df['Price'] = df['Price'].fillna(35.0)
```

In [ ]:
# 数据处理：获得每单总价
df['Total_Price'] = df['Price'] * df['Quantity'] # 向量化操作，底层实现比循环高效很多
df

## Step 4: 数据聚合与分析

In [ ]:
print("任务 A: 数据聚合 - 按分店统计总销售额")
display(df.groupby('Branch')['Total_Price'].sum())
# display(df.groupby('Branch'))

In [ ]:
print("任务 B: 数据透视表 - 每个分店每种产品的总销售额")
( # 处理流太长了可以这样写，函数内参数也是一样
    df.groupby(['Branch', 'Product'], as_index=False) # 使用as_index=False可以避免多级索引（也就是两个列名组合的元组作为行索引）
        .agg(Total_Price=('Total_Price', 'sum'))
        .sort_values(['Branch', 'Total_Price'], ascending=[True, False])
)

### agg() 函数介绍

`agg()` 函数实际上和数据库中学过的聚合函数非常类似  
那么我们一般会遵循：“分组(groupby) -> 聚合(agg)” 的思路来进行数据的汇总与分析  

常用用法：  

- 单列单函数：  
    `.agg(Total_Price=('Total_Price', 'sum'))`
- 多列多函数：（推荐“命名聚合”方式）  
    `.agg(NewCol=('col', 'func'), ...)`
    支持多个NewCol，分别对应各自的列col和函数func
- 多函数列表：  
    `.agg({'col': ['func1', 'func2', ...]})`
    这种方式会生成多级列名  

func 用字符串时代表 Pandas 内置的聚合函数名。常见的有：

- sum, mean, median
- min, max, prod
- count, size, nunique
- std, var, sem
- first, last
- quantile, skew, kurt

也可以传入自定义函数或 numpy 函数。

In [ ]:
df.groupby('Branch', as_index=False).agg(
    Total_Price_Sum=('Total_Price', 'sum'),
    Total_Price_Mean=('Total_Price', 'mean'),
    Quantity_Sum=('Quantity', 'sum')
)

In [ ]:
df.groupby('Branch').agg({'Total_Price': ['sum', 'mean']})

### `pivot_table()`: 数据透视表的另外一种方式
`pivot_table()` 函数可以看作是 `groupby()` + `agg()` 的简化版  

**使用介绍：**

用于把数据"透视"成二维表格（行×列），并对交叉区域做聚合。  
本质上是 groupby + agg + reshape 的快捷写法。

**关键参数：**

- `index`：行标签（分组行）
- `columns`：列标签（分组列）
- `values`：要汇总的数值列
- `aggfunc`：聚合函数（如 sum、mean）
- `fill_value`：空值填充

**实践规范：**

- `values` 只放数值列，避免混合类型导致异常。
- `aggfunc` 明确指定（不要依赖默认 mean）。
- 需要整洁输出时使用 `fill_value` 补 0。
- 结果用于报表时,优先 `pivot_table`；用于进一步计算时,优先 `groupby` + `agg`。

In [ ]:
df.pivot_table(
    index='Branch',      # 行标签
    columns='Product',   # 列标签 (这样产品名会横向展开，对比更清晰)
    values='Total_Price',
    aggfunc='sum',
    fill_value=0         # 没卖出的填0
)

## Step 5: 新数据准备

In [ ]:
# 生成产品价格表 (Master Table)
# 这是一个标准的“字典表”
df_products = pd.DataFrame({
    'Product': ['Latte', 'Espresso', 'Cappuccino', 'Mocha'],
    'Base_Price': [30.0, 25.0, 32.0, 35.0],  # 摩卡的官方价格是 35
    'Category': ['Milk Coffee', 'Black Coffee', 'Milk Coffee', 'Chocolate Coffee']
})

print("产品主数据 (df_products):")
display(df_products)

In [ ]:
# 为了演示时间分析，我们需要稍微“篡改”一下之前的 df，让它包含不同的日期
# 假设这 9 笔订单发生在不同的 3 天里
df['Date'] = ['2023-10-01', '2023-10-01', '2023-10-01', 
              '2023-10-02', '2023-10-02', '2023-10-02', # 10-02 是周一
              '2023-10-07', '2023-10-07', '2023-10-07'] # 10-07 是周六
print("\n更新后的订单表 (Date 已修改):")
display(df.head(3))

## Step 6: 关联数据

In [ ]:
df_merged = pd.merge(df, df_products, how='left', on='Product')

print("\n关联后的订单表 (df_merged):")
display(df_merged)

In [ ]:
# 更改price模拟错误
df_merged.loc[df_merged['Transaction_ID'] == 105, 'Price'] = 33.0

# 同步更新派生列
df_merged['Total_Price'] = df_merged['Price'] * df_merged['Quantity']

df_merged

In [ ]:
# 验证数据完整性
mask_error = df_merged['Price'] != df_merged['Base_Price'] # 此处mask_error也是Series，不过是布尔类型，与表同长（及rows数量相同）
if mask_error.any():
    print("发现价格不匹配的记录:")
    display(df_merged[mask_error]) # 布尔索引筛选，只保留True的行
else:
    print("所有价格均匹配。")

In [ ]:
# 如果考虑修复数据
mask_error = df_merged['Price'] != df_merged['Base_Price']
df_merged.loc[mask_error, 'Price'] = df_merged.loc[mask_error, 'Base_Price']

# 同步更新派生列（只更新错误行）
df_merged.loc[mask_error, 'Total_Price'] = (
    df_merged.loc[mask_error, 'Price'] * df_merged.loc[mask_error, 'Quantity']
)

display(df_merged)

## Step 7: 时间序列处理

In [ ]:
# 1. 转换类型 Parsing
df_merged['Date'] = pd.to_datetime(df_merged['Date']) # 直接替换为日期时间类型的Series

# 2. 提取特征
# .dt为datetime类型专属访问器
df_merged['Weekday_Name'] = df_merged['Date'].dt.day_name() # 星期的全名
df_merged['Is_Weekend'] = df_merged['Date'].dt.weekday >= 5  # Saturday=5, Sunday=6

print("\n含时间特征的订单表 (df_merged):")
print("\nDate 列的数据类型：", df_merged['Date'].dtype)
display(df_merged[['Date', 'Weekday_Name', 'Is_Weekend']].head()) # 注意有两中括号，表示显示内层列表中的几个列

## Step 8: 进阶分析

In [ ]:
print("分析1：周末和工作日的平均客单量")
report_weekend = df_merged.pivot_table(
    index='Is_Weekend',
    values='Total_Price',
    aggfunc=['sum', 'mean', 'count'] # type: ignore
)
display(report_weekend)

In [ ]:
print("\n分析 2: 各产品类别的销售占比")
category_sales = df_merged.groupby('Category')['Total_Price'].sum()
display(category_sales)

## Step 9: 趋势分析

### Pandas 原生写法

In [ ]:
# 按日期整合数据
daily_revenue = df_merged.groupby('Date')['Total_Price'].sum()

# 绘图
plt.figure(figsize=(10, 5)) # 设置画布大小
daily_revenue.plot(
    kind='line',
    marker='o',
    color='tab:blue',
    linestyle='--'
)

plt.title('Daily Revenue Trend', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Revenue (CNY)')
plt.show()